In [1]:

import os
import time
import oss2
import json

import cv2 as cv
import numpy as np
from tqdm import tqdm
from sko.GA import GA

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkfacebody.request.v20191230.CompareFaceRequest import CompareFaceRequest


In [2]:

access_key_id = os.getenv('OSS_TEST_ACCESS_KEY_ID', '')
access_key_secret = os.getenv('OSS_TEST_ACCESS_KEY_SECRET', '')
bucket_name = os.getenv('OSS_TEST_BUCKET', 'ivan-bucket-out-002')
endpoint = os.getenv('OSS_TEST_ENDPOINT', 'oss-cn-shanghai.aliyuncs.com')

bucket = oss2.Bucket(oss2.Auth(access_key_id, access_key_secret), endpoint, bucket_name)

client = AcsClient(access_key_id, access_key_secret, 'cn-shanghai')


In [3]:

#
def _f1(imgA=None, imgB=None, Rect=None):
    while True:
        try:
            time.sleep(0.68)
            if imgA:
                oss2.resumable_upload(bucket, 'imgA.jpg', imgA)
            if imgB:
                oss2.resumable_upload(bucket, 'imgB.jpg', imgB)

            request = CompareFaceRequest()
            request.set_accept_format('json')

            request.set_ImageURLA(f"https://{bucket_name}.{endpoint}/imgA.jpg")
            request.set_ImageURLB(f"https://{bucket_name}.{endpoint}/imgB.jpg")

            response = client.do_action_with_exception(request)
            
            if Rect:
                return json.loads(response)["Data"]["Confidence"], json.loads(response)["Data"]["RectAList"]
            else:
                return json.loads(response)["Data"]["Confidence"]
        except Exception as e:
            print(e)

#
def _f2(imgA, imgB):
    imgA, imgB = cv.imread(imgA), cv.imread(imgB)
    return np.sqrt(np.sum(np.square(imgA - imgB)))


In [4]:
%time _f1(imgA='../Data/Z0.jpg', imgB='../Data/Z0.jpg')

CPU times: user 59.7 ms, sys: 23.9 ms, total: 83.7 ms
Wall time: 1.15 s


100.0

In [5]:
%time _f1(imgA='../Data/Z0.jpg', Rect=1)

CPU times: user 6.31 ms, sys: 2.02 ms, total: 8.33 ms
Wall time: 1.09 s


(100.0, [19, 22, 77, 89])

In [6]:
%time _f1(imgA='../Data/Z0.jpg')

CPU times: user 5 ms, sys: 1.65 ms, total: 6.65 ms
Wall time: 953 ms


100.0

In [7]:
%time _f1(imgA='../Data/Z0.jpg', imgB='../Data/Z1.jpg')

CPU times: user 7.77 ms, sys: 2.22 ms, total: 9.99 ms
Wall time: 986 ms


45.32559585571289

In [8]:
%time _f1(imgA='../Data/bad.png', imgB='../Data/bad.png')

CPU times: user 8.61 ms, sys: 2.62 ms, total: 11.2 ms
Wall time: 1.23 s


0.0

In [9]:
%time _f2(imgA='../Data/Z0.jpg', imgB='../Data/Z1.jpg')

CPU times: user 1.61 ms, sys: 1.76 ms, total: 3.38 ms
Wall time: 1.74 ms


768.1659976853962

In [10]:
%time _f2(imgA='../Data/Z0.jpg', imgB='../Data/Z0.jpg')

CPU times: user 1.34 ms, sys: 1.38 ms, total: 2.72 ms
Wall time: 1.27 ms


0.0

In [11]:

pNum = "T000001"

proj = ".."
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"
pout = f"{proj}/Out/{pNum}"

os.system(f"rm -rf {pout}/temp*")
os.system(f"rm -rf {pout}/images/*")


0

In [12]:
# ! rm -rf ../Out/T000001/images/*

In [13]:

st, ed = 1, 1
N, _N, M = 112, 128, 2# [2, 4, 8, 16, 32, 64, 128]
# N, _N, M = 112, 112, 112
S = M**2*3

pl = lambda x: sorted(
    [
        (i-_N//x, i, j-_N//x, j, k)
        for j in range(0, _N+1, _N//x) if j > 0
        for i in range(0, _N+1, _N//x) if i > 0
        for k in range(3)
     ]
)
dpl = pl(M)


In [15]:

RUN = 2
BIG = 25
CUT = 50

for i in tqdm(range(st, ed+1), desc="All"):
    mscore = 0
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    tp = f"{pout}/temp.jpg"
    
    mescore, [x, y, w, h] = _f1(imgA=pi, imgB=pi, Rect=1)
    min_y, max_y, min_x, max_x = y, y+h, x, x+w
    print(mescore, min_y, max_y, min_x, max_x)
    if mescore == 0:# or os.path.exists(op):
        print(pi, "%.6f"%mescore)
        continue
    
    imgb = cv.imread(pi)
    score = {}
    scmin = [99999]
    # 
    def _function(X):
        _X = X.tostring()
        
        if _X in score:
            return score[_X]
        else:
            imgo1 = imgb.copy()
            imgo2 = imgb.copy()
            for nX, lX in enumerate(X):
                idpl = dpl[nX]
                imgo1[idpl[0]:idpl[1], idpl[2]:idpl[3], idpl[4]] = \
                imgo1[idpl[0]:idpl[1], idpl[2]:idpl[3], idpl[4]] + lX
            # imgo2[min_y:max_y, min_x:max_x] = imgo1[min_y:max_y, min_x:max_x]
            cv.imwrite(tp, imgo1)
            
            _s1 = _f1(imgB=tp)
            _s2 = _f2(imgA=pi, imgB=tp)

            _s1 = CUT if _s1 <= CUT else _s1
            _s2 = _s2
            _s_ = (2*_s1*_s2)/(_s1+_s2)
            _s_ = _s1
            
            if _s_ < min(scmin):
                scmin.append(_s_)
            score[_X] = _s_
            
            print("Min:%.2f, Avg:%.6f, S1:%.6f, S2:%.6f, SC:%.6f" % (min(scmin), np.mean(np.abs(X)), _s1, _s2, _s_))
            return _s_
    
    try:
        _lb, _ub = [], []
        for nX, lX in enumerate([1 for _ in range(S)]):
            idpl = dpl[nX]
            try:
                _min = np.min(imgb[idpl[0]:idpl[1], idpl[2]:idpl[3], idpl[4]])
                _max = np.max(imgb[idpl[0]:idpl[1], idpl[2]:idpl[3], idpl[4]])
                _min = max(_min-BIG, 0) - _min
                _max = min(_max+BIG, 255) - _max
            except:
                _min = 0
                _max = 1
            _lb.append(_min)
            _ub.append(_max)
        # print(_lb, _ub)
        
        """
        入参|默认值|意义
        func|-|目标函数
        n_dim|-|目标函数的维度
        size_pop|50|种群规模
        max_iter|200|最大迭代次数
        prob_mut|0.001|变异概率
        lb|-1|每个参数的最小值
        ub|1|每个参数的最大值
        constraint_eq|空元组|线性约束
        constraint_ueq|空元组|非线性约束
        precision|1e-7|精准度，int/float或者它们组成的列表
        """
        # from sko.DE import DE as GA
        ga = GA(
            func=_function,
            n_dim=S,
            size_pop=10,
            max_iter=RUN,
            prob_mut=0.1,
            lb=_lb, 
            ub=_ub,
            precision=[1 for _ in range(S)]
        )
        best_x, best_y = ga.run()
        # ga.Chrom = np.random.randint(0, 1, size=(10, S))
    except Exception as e:
        print(e)
    
#     X = best_x
#     imgb = cv.imread(pi)
#     imgo = imgb.copy()
#     for nX, lX in enumerate(X):
#         idpl = dpl[nX]
#         imgo[idpl[0]:idpl[1], idpl[2]:idpl[3], idpl[4]] = \
#         imgo[idpl[0]:idpl[1], idpl[2]:idpl[3], idpl[4]] - lX
#     imgo[min_y:max_y, min_x:max_x] = imgb[min_y:max_y, min_x:max_x]
#     # imgo = np.clip(imgo, imgb-BIG, imgb+BIG)
#     cv.imwrite(f"{op}", imgo)
#     print(">>>", pi, "%.6f"%_f1(imgA=f"{pi}", imgB=f"{op}"), "%.6f"%_f2(imgA=f"{pi}", imgB=f"{op}"))




All:   0%|          | 0/1 [00:00<?, ?it/s]

99.99996185302734 17 111 19 103


/Users/ivan/Desktop/ALL/Soft/python3/lib/python3.7/site-packages/sko/GA.py:179: RuntimeWarning: invalid value encountered in true_divide
  return (b * mask).sum(axis=1) / mask.sum()
/Users/ivan/Desktop/ALL/Soft/python3/lib/python3.7/site-packages/sko/GA.py:193: RuntimeWarning: invalid value encountered in greater
  X = np.where(X > self.ub, self.ub, X)


Min:84.50, Avg:nan, S1:84.503777, S2:2018.239579, SC:84.503777
Min:84.00, Avg:nan, S1:83.995590, S2:1567.581258, SC:83.995590
Min:83.94, Avg:nan, S1:83.936256, S2:1624.214887, SC:83.936256
Min:83.94, Avg:nan, S1:84.459381, S2:2037.919037, SC:84.459381
Min:83.10, Avg:nan, S1:83.096115, S2:1879.240538, SC:83.096115
Min:83.10, Avg:nan, S1:84.583038, S2:1744.564989, SC:84.583038
Min:83.10, Avg:nan, S1:83.415085, S2:1652.683878, SC:83.415085
Min:83.10, Avg:nan, S1:84.068680, S2:1639.848164, SC:84.068680
Min:83.10, Avg:nan, S1:83.794914, S2:1847.462855, SC:83.794914
Min:83.10, Avg:nan, S1:83.402946, S2:1865.752127, SC:83.402946
Min:83.10, Avg:nan, S1:85.272392, S2:1636.832612, SC:85.272392
Min:83.10, Avg:nan, S1:84.546555, S2:1689.517683, SC:84.546555
Min:83.10, Avg:nan, S1:83.285149, S2:1727.783262, SC:83.285149
Min:82.07, Avg:nan, S1:82.069321, S2:1643.760932, SC:82.069321
Min:82.07, Avg:nan, S1:83.316734, S2:1589.092508, SC:83.316734
Min:82.07, Avg:nan, S1:83.140511, S2:1883.549840, SC:83


All: 100%|██████████| 1/1 [00:20<00:00, 20.24s/it]


Min:82.07, Avg:nan, S1:83.482719, S2:1622.078605, SC:83.482719


In [ ]:

# st, el = 1, 712

# savl = []
# for i in tqdm(range(st, el+1)):
#     pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
#     op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
#     if not os.path.exists(op):
#         img = cv.imread(pi)
#         cv.imwrite(op, img)
#         savl.append(1)
# print(len(savl))


In [ ]:
# ! sh run.sh